In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import pandas as pd 
from selenium.webdriver.common.action_chains import ActionChains
import re

In [ ]:
# 아마존 기본 URL
base_url = 'https://www.amazon.com'

# 각 제품별 리뷰페이지(최신리뷰 순으로 정렬) URL
filter_url  = '&reviewerType=all_reviews&sortBy=recent&pageNumber=1'

# 카테고리별로 총 10페이지
li = [str(i) for i in range(1,11)]

# 카테고리별 URL

# Arts and Crafts
fir_url = 'https://www.amazon.com/s?i=arts-crafts-intl-ship&rh=n%3A4954955011&s=featured-rank&page='
sec_url = '&qid=1604564748&ref=sr_pg_'

# Automotive
# fir_url = 'https://www.amazon.com/s?i=automotive-intl-ship&rh=n%3A2562090011&s=featured-rank&page='
# sec_url = '&qid=1604564796&ref=sr_pg_'

# Baby
# fir_url = 'https://www.amazon.com/s?i=baby-products-intl-ship&rh=n%3A16225005011&s=featured-rank&page='
# sec_url = '&qid=1604564842&ref=sr_pg_'

# Beauty and Personal Care 
# fir_url = 'https://www.amazon.com/s?i=beauty-intl-ship&rh=n%3A16225006011&s=featured-rank&page='
# sec_url = '&qid=1604553387&ref=sr_pg_'

# Health and Household
# fir_url = 'https://www.amazon.com/s?i=hpc-intl-ship&rh=n%3A16225010011&s=featured-rank&page='
# sec_url = '&qid=1604560064&ref=sr_pg_'

# Home and Kitchen
# fir_url = 'https://www.amazon.com/s?i=kitchen-intl-ship&bbn=16225011011&rh=n%3A16225011011%2Cn%3A284507&dc&page='
# sec_url = '&fst=as%3Aoff&pf_rd_i=16225011011&pf_rd_m=ATVPDKIKX0DER&pf_rd_p=710a58c3-24eb-4d05-9fcb-f0473b95be15&pf_rd_r=HS1KHPHF1SPC37STXTCQ&pf_rd_s=merchandised-search-3&pf_rd_t=101&qid=1487191426&rnid=16225011011&ref=sr_pg_'

# Pet Supplies
# fir_url = 'https://www.amazon.com/s?i=pets-intl-ship&rh=n%3A16225013011&s=featured-rank&page='
# sec_url = '&qid=1604564116&ref=sr_pg_'

# Sports and Outdoors
# fir_url = 'https://www.amazon.com/s?i=sporting-intl-ship&rh=n%3A16225014011&s=featured-rank&page='
# sec_url = '&qid=1604564227&ref=sr_pg_'

# Tools and Home Improvement
# fir_url = 'https://www.amazon.com/s?i=tools-intl-ship&rh=n%3A256643011&s=featured-rank&page='
# sec_url = '&qid=1604564283&ref=sr_pg_'

# Toys and Games
# fir_url = 'https://www.amazon.com/s?i=toys-and-games-intl-ship&rh=n%3A16225015011&s=featured-rank&page='
# sec_url = '&qid=1604564380&ref=sr_pg_'


# 데이터 들어갈 빈 리스트
urls = []
comment = []
satisfaction = []

df_url = []
df_comment = []
df_satisfaction = []

# 카테고리별 10페이지씩 넘기면서 모든 제품들의 리뷰페이지 URL 주소 받아오기
for i in range(len(li)) :
    # 크롬드라이버로 페이지 열기
    driver = webdriver.Chrome('chromedriver.exe')
    driver.get(fir_url + li[i] + sec_url + li[i])
    time.sleep(2.0)

    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'lxml')
    prods = soup.select('#search > div > div > div > span:nth-child(4) > div > div > div > span > div > div > div:nth-child(3) > h2 > a > span')
    
    for i in range(len(prods)): 
        urls.append(base_url + 
                    soup.select('#search > div > div > div > span:nth-child(4) > div > div > div > span > div > div > div:nth-child(3) > h2 > a')[i]['href']\
                    .replace('dp', 'product-reviews').replace('product_top', 'viewopt_srt') + filter_url)    
    driver.close()



# 총 URL 갯수만큼 리뷰 크롤링 실행
for i in urls :
    driver = webdriver.Chrome('chromedriver.exe')
    driver.get(i)
    this_url = i
    
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'lxml')

    star = soup.select('div.a-row > a > i > span.a-icon-alt')
    reply = soup.select('div.a-row.a-spacing-small.review-data > span.a-size-base.review-text.review-text-content > span')
    
    # 가져온 별점의 수와 댓글의 수가 일치하면 가져오고 아니면 다음 URL로 이동
    if len(star) == len(reply) :
        for i in range(len(star)) :
            satisfaction.append(star[i].text.split(' ')[0])
            comment.append(reply[i].text.replace('\n', '').replace('\t', ''))

        # 댓글페이지 100장 넘겨서 제품별로 총 1000개 댓글 가져오기
        for i in range(100) :
            time.sleep(2.5)
            try :
                driver.find_element_by_xpath('//*[@id="cm_cr-pagination_bar"]/ul/li[2]').click()
                html_source = driver.page_source
                soup = BeautifulSoup(html_source, 'lxml')

                star = soup.select('div.a-row > a > i > span.a-icon-alt')
                reply = soup.select('div.a-row.a-spacing-small.review-data > span.a-size-base.review-text.review-text-content > span')
                
                # 댓글페이지가 넘어가면서도 갯수가 맞지 않으면 그대로 다음 URL로 건너뜀
                if len(star) == len(reply) :
                    for i in range(len(star)) :
                        satisfaction.append(star[i].text.split(' ')[0])
                        comment.append(reply[i].text.replace('\n', '').replace('\t', ''))
                else :
                    print(this_url)
                    continue
            except :
                pass
    else :
        print(this_url)
        driver.close()
        continue
    print(this_url)
    driver.close()

# 데이터프레임 만들고 csv 로 저장 
amazon_df = pd.DataFrame({
    "reviews" : comment,
    "label" : satisfaction
})
amazon_df.dropna(inplace = True)
print(amazon_df)

amazon_df.to_csv('Arts and Crafts.csv', index = False, encoding = 'utf-8')
# amazon_df.to_csv('Automotive.csv', index = False, encoding = 'utf-8')
# amazon_df.to_csv('Baby.csv', index = False, encoding = 'utf-8')
# amazon_df.to_csv('Beauty and Personal Care.csv', index = False, encoding = 'utf-8')
# amazon_df.to_csv('Health and Household.csv', index = False, encoding = 'utf-8')
# amazon_df.to_csv('Home Kitchen.csv', index = False, encoding = 'utf-8')
# amazon_df.to_csv('Pet Supplies.csv', index = False, encoding = 'utf-8')
# amazon_df.to_csv('Sports and Outdoors.csv', index = False, encoding = 'utf-8')
# amazon_df.to_csv('Tools and Home Improvement.csv', index = False, encoding = 'utf-8')
# amazon_df.to_csv('Toys and Games.csv', index = False, encoding = 'utf-8')